In [1]:
import random
import numpy as np
import math

In [2]:
load = [18, 5, 5, 5, 5, 1, 7, 3, 4, 22, 24, 25, 26, 27, 28, 27, 23, 21, 32, 27, 34, 24, 25, 20]
sum(load)/len(load)

18.25

In [3]:
wind = [random.randint(2,15) for _ in range(24)]
#wind

In [19]:
# Generate 12 points between 0 and pi (half of the sine wave)
x = np.linspace(0, np.pi, 12)

# Apply the sine function to get a half-sinusoidal distribution
sinusoidal_values = np.sin(x)

# Normalize the values to a specific range if needed (optional)
# For example, scaling between 0 and 1
normalized_values = sinusoidal_values / np.max(sinusoidal_values)

# Peak Solar power is 20KW
solar = np.round(normalized_values * 20)
solar = [0] * 6 + solar.tolist() + [0] * 6
#solar

regenerate = [a + b for a, b in zip(wind, solar)]

In [20]:
print(f'Hourly regenerative power available is: {regenerate}')

Hourly load for a day is: [18, 5, 5, 5, 5, 1, 7, 3, 4, 22, 24, 25, 26, 27, 28, 27, 23, 21, 32, 27, 34, 24, 25, 20]
Hourly regenerative power available is: [12, 9, 9, 14, 5, 11, 11.0, 9.0, 25.0, 20.0, 26.0, 29.0, 24.0, 20.0, 19.0, 25.0, 8.0, 11.0, 6, 13, 15, 14, 7, 3]


In [54]:
print(f'Hourly load for a day is: {load}')
# Base load generator
nuclear = 6

# Total green power generation available as sum of wind, solar, nuclear
available_supply = [green + nuclear for green in regenerate]
print(f'Available green power in microgrid is: {available_supply}')

# Power deficit is load minus available power
deficit_power = [a - b for a,b in zip(load, available_supply)]
print(f'Deficit in microgrid is: {deficit_power}')

# Initialize battery requirement
soc = sum(available_supply) - sum(load)
battery_capacity = 0
#cap = []

for reqd_power in deficit_power:
    if reqd_power < 0:
        soc -= reqd_power
    else:  # Charge the battery (excess in power)
        soc = max(soc - reqd_power, 0)  # Ensure no negative charge
    # Track the maximum battery capacity needed
    battery_capacity = max(battery_capacity, soc)
    #cap.append(soc)

battery_capacity -= sum(available_supply) - sum(load)
# Output the maximum battery capacity required
print(f'The battery capacity required is: {battery_capacity} kWh')
#print(cap)

Hourly load for a day is: [18, 5, 5, 5, 5, 1, 7, 3, 4, 22, 24, 25, 26, 27, 28, 27, 23, 21, 32, 27, 34, 24, 25, 20]
Available green power in microgrid is: [18, 15, 15, 20, 11, 17, 17.0, 15.0, 31.0, 26.0, 32.0, 35.0, 30.0, 26.0, 25.0, 31.0, 14.0, 17.0, 12, 19, 21, 20, 13, 9]
Deficit in microgrid is: [0, -10, -10, -15, -6, -16, -10.0, -12.0, -27.0, -4.0, -8.0, -10.0, -4.0, 1.0, 3.0, -4.0, 9.0, 4.0, 20, 8, 13, 4, 12, 11]
The battery capacity required is: 132.0 kWh
[51.0, 61.0, 71.0, 86.0, 92.0, 108.0, 118.0, 130.0, 157.0, 161.0, 169.0, 179.0, 183.0, 182.0, 179.0, 183.0, 174.0, 170.0, 150.0, 142.0, 129.0, 125.0, 113.0, 102.0]


In [46]:
print(f'Total load in a day: sum(load)')
print(f'Total generation in a day: sum(available_supply)')
print(f'Total of power not consumed: sum(wasted_power)')

Total load in a day: sum(load)
Total generation in a day: sum(available_supply)
Total of power not consumed: sum(wasted_power)


In [59]:
# Introduce Storage to cater for deficit
print(f'Required Battery capacity: {battery_capacity}')
battery_charged = sum(available_supply) - sum(load)
soc = [round(battery_charged / battery_capacity, 2)]

wasted_power = []

for deficit in deficit_power:
    if deficit > 0:
        battery_charged -= deficit
        wasted_power.append(0)
    else:
        battery_charged -= deficit
        if battery_charged >= battery_capacity:
            wasted_power.append(battery_charged - battery_capacity)
            battery_charged = battery_capacity
        
    soc.append(round(battery_charged / battery_capacity, 2))

print(f'State of Battery Charge: {soc}')
print(f'Energy Wasted: {wasted_power}')

Required Battery capacity: 132.0
State of Battery Charge: [0.39, 0.39, 0.46, 0.54, 0.65, 0.7, 0.82, 0.89, 0.98, 1.0, 1.0, 1.0, 1.0, 1.0, 0.99, 0.97, 1.0, 0.93, 0.9, 0.75, 0.69, 0.59, 0.56, 0.47, 0.39]
Energy Wasted: [25.0, 4.0, 8.0, 10.0, 4.0, 0, 0, 0.0, 0, 0, 0, 0, 0, 0, 0, 0]


In [61]:
print(sum(regenerate))
print(sum(load))
print(sum(wasted_power))

345.0
438
51.0


In [6]:
print(sum(regenerate))
print(sum(load))

417.0
438


In [7]:
# Calculate net load (Load - Regenerative Power Available)
net_load = [a - b for a, b in zip(regenerate,load)]
print(net_load)

# Initialize battery requirement
soc = 0
battery_capacity = 0

for reqd_power in net_load:
    if reqd_power > 0:
        soc += reqd_power
    else:  # Charge the battery (excess in power)
        soc = max(soc + reqd_power, 0)  # Ensure no negative charge
    # Track the maximum battery capacity needed
    battery_capacity = max(battery_capacity, soc)

# Output the maximum battery capacity required
print(f'The battery capacity required is: {battery_capacity} kWh')


[-6, 4, 4, 9, 0, 10, 4.0, 9.0, 26.0, 6.0, 12.0, 14.0, 8.0, 3.0, -1.0, 3.0, -12.0, -10.0, -26, -14, -19, -10, -18, -17]
The battery capacity required is: 111.0 kWh


In [9]:
import numpy as np

# Hourly load and regenerative power data
hourly_load = np.array(load)
regenerative_power = np.array(regenerate)

# Calculate net load
net_load = np.maximum(hourly_load - regenerative_power, 0)

# Determine the maximum net load
max_net_load = np.max(net_load)

print(f"The maximum net load is: {max_net_load} units")


The maximum net load is: 26.0 units


In [10]:
import numpy as np

# Hourly load and regenerative power data
hourly_load = np.array([18, 5, 5, 5, 5, 1, 7, 3, 4, 22, 24, 25, 26, 27, 28, 27, 23, 21, 32, 27, 34, 24, 25, 20])
regenerative_power = np.array([7, 10, 2, 2, 10, 15, 6.0, 37.0, 66.0, 86.0, 102.0, 102.0, 102.0, 105.0, 88.0, 62.0, 40.0, 7.0, 8, 15, 13, 8, 2, 12])

# Base load generator capacity (in kW)
base_generator_capacity = 10  # kW

# Calculate energy deficit
energy_deficit = np.maximum(hourly_load - regenerative_power, 0)

# Calculate total energy deficit (in kWh)
total_energy_deficit = np.sum(energy_deficit)  # Since each value is in kW and the duration is 1 hour

print(f"Total energy deficit for the day: {total_energy_deficit} kWh")

# Optional: You can also calculate the battery capacity considering the generator capacity
battery_capacity = total_energy_deficit  # Assuming the battery needs to cover the total deficit
print(f"Required battery capacity: {battery_capacity} kWh")


Total energy deficit for the day: 136.0 kWh
Required battery capacity: 136.0 kWh


In [11]:
import numpy as np

# Hourly load and regenerative power data
hourly_load = np.array([18, 5, 5, 5, 5, 1, 7, 3, 4, 22, 24, 25, 26, 27, 28, 27, 23, 21, 32, 27, 34, 24, 25, 20])
regenerative_power = np.array([7, 10, 2, 2, 10, 15, 6.0, 37.0, 66.0, 86.0, 102.0, 102.0, 102.0, 105.0, 88.0, 62.0, 40.0, 7.0, 8, 15, 13, 8, 2, 12])

# Base load generator capacities
min_generator_output = 6  # kW
max_generator_output = 10  # kW

# Calculate energy deficit
energy_deficit = np.maximum(hourly_load - regenerative_power, 0)

# Calculate the energy contribution of the generator
generator_contribution = np.minimum(max_generator_output, energy_deficit)

# Adjust the generator output to cover the deficit within its range
remaining_deficit = np.maximum(energy_deficit - generator_contribution, 0)

# Calculate total remaining energy deficit (in kWh)
total_remaining_deficit = np.sum(remaining_deficit)  # Since each value is in kW and the duration is 1 hour

print(f"Total remaining energy deficit for the day: {total_remaining_deficit} kWh")

# Battery capacity is the remaining deficit that needs to be covered
battery_capacity = total_remaining_deficit
print(f"Required battery capacity: {battery_capacity} kWh")


Total remaining energy deficit for the day: 51.0 kWh
Required battery capacity: 51.0 kWh


In [12]:
import numpy as np

# Hourly load and regenerative power data
hourly_load = np.array([18, 5, 5, 5, 5, 1, 7, 3, 4, 22, 24, 25, 26, 27, 28, 27, 23, 21, 32, 27, 34, 24, 25, 20])
regenerative_power = np.array([7, 10, 2, 2, 10, 15, 6.0, 37.0, 66.0, 86.0, 102.0, 102.0, 102.0, 105.0, 88.0, 62.0, 40.0, 7.0, 8, 15, 13, 8, 2, 12])

# Base load generator capacities
min_generator_output = 6  # kW
max_generator_output = 10  # kW

# Calculate energy deficit
energy_deficit = np.maximum(hourly_load - regenerative_power, 0)

# Calculate the generator's actual contribution for each hour
generator_contribution = np.minimum(max_generator_output, energy_deficit)
# Adjust generator output within its range
generator_contribution = np.maximum(generator_contribution, min_generator_output)

# Calculate the remaining deficit after accounting for the generator's contribution
remaining_deficit = np.maximum(energy_deficit - generator_contribution, 0)

# Calculate total remaining energy deficit (in kWh)
total_remaining_deficit = np.sum(remaining_deficit)  # Since each value is in kW and the duration is 1 hour

print(f"Total remaining energy deficit for the day: {total_remaining_deficit} kWh")

# Battery capacity is the remaining deficit that needs to be covered
battery_capacity = total_remaining_deficit
print(f"Required battery capacity: {battery_capacity} kWh")


Total remaining energy deficit for the day: 51.0 kWh
Required battery capacity: 51.0 kWh
